In [1]:
import sys
import datetime
import argparse
import tables   as tb
import numpy    as np
import pandas   as pd

import antea.database.load_db    as db
import antea.reco.reco_functions as rf
import antea.elec.tof_functions  as tf

from antea.io   .mc_io           import load_mchits
from antea.io   .mc_io           import load_mcparticles
from antea.io   .mc_io           import load_mcsns_response
from antea.io   .mc_io           import load_mcTOFsns_response

In [2]:
eventsPath = '/Users/carmenromoluque/Desktop/check_neutrinos_type_uint/'
file_name  = 'petit_ring_tof_all_tables'


DataSiPM     = db.DataSiPM('petalo', 0)
DataSiPM_idx = DataSiPM.set_index('SensorID')


### TOF elec parameters:
SIPM        = {'n_sipms':3500, 'first_sipm':1000, 'tau_sipm':[100,15000]}
n_sipms     = SIPM['n_sipms']
first_sipm  = SIPM['first_sipm']
tau_sipm    = SIPM['tau_sipm']
TE_range    = [0.25]
TE_TDC      = TE_range[0]
time_window = 10000
time_bin    = 5
time        = np.arange(0, 80000, time_bin)
#spe_resp, norm    = tf.spe_dist(tau_sipm, time)
spe_resp, norm    = tf.apply_spe_dist(time) 

for number in range(0, 1):
    number_str = "{:03d}".format(number)
    filename  = f"{eventsPath}/{file_name}.{number_str}.pet.h5"
    sns_response = load_mcsns_response(filename)

    sns_response_tof = load_mcTOFsns_response(filename)
    particles        = load_mcparticles(filename)
    hits             = load_mchits(filename)

    events = particles.event_id.unique()

    sipms         = DataSiPM_idx.loc[sns_response.sensor_id]
    sns_ids       = sipms.index.values
    sns_positions = np.array([sipms.X.values, sipms.Y.values, sipms.Z.values]).transpose()

    charge_range = (1000, 1400)

    for evt in events[31:32]:
        #for evt in events[:]:
        evt_sns = sns_response[sns_response.event_id == evt]
        evt_sns = rf.find_SiPMs_over_threshold(evt_sns, threshold=2)
        if len(evt_sns) == 0:
            continue

        evt_parts = particles[particles.event_id       == evt]
        evt_hits  = hits[hits.event_id                 == evt]
        evt_tof   = sns_response_tof[sns_response_tof.event_id == evt]
        
        
        

In [57]:
def tdc_convolution(tof_response, spe_response, time_window, s_id, te_tdc):
    pe_vect = np.zeros(time_window)
    sel_tof = tof_response[(tof_response.sensor_id == s_id)&(tof_response.time_bin < time_window)]
    pe_vect[sel_tof.time_bin.values] = sel_tof.charge.values
    tdc_conv = tf.convolve_tof(spe_response, pe_vect)
    return tdc_conv

In [97]:
def translate_charge_conv_to_wf_df(event_id, s_id, conv_vect):
    keys        = np.array(['event_id', 'sensor_id', 'time_bin', 'charge'])
    t_bin       = np.where(conv_vect>0)[0]
    charge      = conv_vect[conv_vect>0]
    evt         = np.full(len(t_bin), event_id)
    sns_id_full = np.full(len(t_bin), s_id)
    a_wf        = np.array([evt, sns_id_full, t_bin, charge])
    wf_df       = pd.DataFrame(a_wf.T, columns=keys).astype({'event_id': 'int32',
                                                             'sensor_id': 'int32',
                                                             'time_bin' : 'int32'})
    return wf_df

In [ ]:
def function1(evt_tof, spe_resp, time_window, TE_TDC):
    tof_sns = evt_tof.sensor_id.unique()
    evt_tof_exp_dist = pd.DataFrame({})
    for s_id in tof_sns[:]:
        tdc_conv    = tdc_convolution(evt_tof, spe_resp, time_window, s_id, TE_TDC)
        tdc_conv_df = translate_charge_conv_to_wf_df(evt, s_id, tdc_conv)
        evt_tof_exp_dist = evt_tof_exp_dist.append(tdc_conv_df)
    return evt_tof_exp_dist

def function2(evt_tof, spe_resp, time_window, TE_TDC):
    tof_sns = evt_tof.sensor_id.unique()
    evt_tof_exp_dist = []
    for s_id in tof_sns[:]:
        tdc_conv    = tdc_convolution(evt_tof, spe_resp, time_window, s_id, TE_TDC)
        tdc_conv_df = translate_charge_conv_to_wf_df(evt, s_id, tdc_conv)
        evt_tof_exp_dist.append(tdc_conv_df)
    evt_tof_exp_dist = pd.concat(evt_tof_exp_dist)
    return evt_tof_exp_dist

In [ ]:
fn1 = function1(evt, evt_tof, spe_resp, time_window, te_tdc)
fn2 = function2(evt, evt_tof, spe_resp, time_window, te_tdc)
print(fn1==fn2)

### Old functions:

In [ ]:
def tdc_convolution(tof_response: pd.DataFrame,
                    spe_response: Sequence[float],
                    time_window: float,
                    s_id: int,
                    first_sipm: int,
                    te_tdc: float) -> Sequence[Sequence[float]]:
    """
    Apply the spe_response distribution to every sipm and returns a charge
    matrix of time and n_sipms dimensions.
    """
    #pe_table = np.zeros((time_window, n_sipms))
    sel_tof  = tof_response[tof_response.time_bin < time_window]
    s_ids    = - sel_tof.sensor_id.values - first_sipm
    pe_table[sel_tof.time_bin.values] = sel_tof.charge.values
    print(np.count_nonzero(pe_table))
    return pe_table
#conv_table = np.zeros((len(pe_table) + len(spe_response)-1, n_sipms))
    #for i in range(n_sipms):
#    for i in range(10,11):
#        print(convolve_tof(spe_response, pe_table[0:time_window,i]).shape)
#        print(np.count_nonzero(convolve_tof(spe_response, pe_table[0:time_window,i])))
#        if np.count_nonzero(pe_table[:,i]):
#            conv_table[:,i] = convolve_tof(spe_response, pe_table[0:time_window,i])
#return conv_table


def translate_charge_matrix_to_wf_df(event_id: int,
                                     s_id: int,
                                     conv_vect: Sequence[float],
                                     first_sipm: int) -> pd.DataFrame:
    """
    Transform the charge matrix into a tof dataframe.
    """
    keys         = np.array(['event_id', 'sensor_id', 'time_bin', 'charge'])
    if np.all(conv_vect==0):
        return pd.DataFrame({}, columns=keys)
    t_bin       = np.where(conv_vect>0)[0].astype('int32')
    charge      = conv_vect[conv_vect>0]
    evt         = np.full(len(t_bin), event_id)
    sns_id_full = np.full(len(t_bin), s_id)
    a_wf        = np.array([evt, sns_id_full, t_bin, charge])
    wf_df       = pd.DataFrame(a_wf.T, columns=keys).astype({'event_id': 'int32',
                                                             'sensor_id': 'int32',
                                                             'time_bin' : 'int32'})
    return wf_df


In [3]:
sns_response_tof[sns_response_tof.event_id == evt].head()

,event_id,sensor_id,time_bin,charge
29861,31,-4089,667,1
29862,31,-4089,1297,1
29863,31,-4089,1392,1
29864,31,-4089,3441,1
29865,31,-4089,5979,1


In [4]:
sns_sel1 = sns_response_tof[(sns_response_tof.event_id==31)&(sns_response_tof.sensor_id == -4089)]
sns_sel1

,event_id,sensor_id,time_bin,charge
29861,31,-4089,667,1
29862,31,-4089,1297,1
29863,31,-4089,1392,1
29864,31,-4089,3441,1
29865,31,-4089,5979,1
29866,31,-4089,10251,1
29867,31,-4089,13725,1


In [45]:
sns_sel2 = sns_response_tof[(sns_response_tof.event_id==31)&(sns_response_tof.sensor_id == -1740)]
sns_sel2

,event_id,sensor_id,time_bin,charge
32839,31,-1740,970,1
32840,31,-1740,4782,1


In [47]:
pd.concat([sns_sel1, sns_sel2])

,event_id,sensor_id,time_bin,charge
29861,31,-4089,667,1
29862,31,-4089,1297,1
29863,31,-4089,1392,1
29864,31,-4089,3441,1
29865,31,-4089,5979,1
29866,31,-4089,10251,1
29867,31,-4089,13725,1
32839,31,-1740,970,1
32840,31,-1740,4782,1


In [54]:
arr1 = np.array([1,0,2,0,9,8])
np.where(arr1>0)[0].astype('int32')

array([ 0, -2, -4, -5], dtype=int32)

In [99]:
df = pd.DataFrame({})

In [91]:
df.empty

True

In [93]:
keys        = np.array(['event_id', 'sensor_id', 'time_bin', 'charge'])
df = pd.DataFrame({}, columns=keys)

In [94]:
df.empty

True

In [95]:
df = pd.concat([df, sns_sel1])

,event_id,sensor_id,time_bin,charge
29861,31,-4089,667,1
29862,31,-4089,1297,1
29863,31,-4089,1392,1
29864,31,-4089,3441,1
29865,31,-4089,5979,1
29866,31,-4089,10251,1
29867,31,-4089,13725,1


In [102]:
df = df.append(sns_sel1)

,event_id,sensor_id,time_bin,charge
29861,31,-4089,667,1
29862,31,-4089,1297,1
29863,31,-4089,1392,1
29864,31,-4089,3441,1
29865,31,-4089,5979,1
29866,31,-4089,10251,1
29867,31,-4089,13725,1


In [9]:
sns_sel1.event_id.dtype == 'int32'
sns_sel1.sensor_id.dtype

True